# Deep convolutional  GANs

<div>
   <span style="font-size: large;"> "GANS in Action"</span>, Jakub Langr, Vladimir Bok, Manning Publications 2019.  
</div>

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import itertools
from IPython.display import clear_output, display

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import torch as t
import torch.nn as tnn
import torch.nn.functional as F

import torchvision

In [ ]:
import sys
sys.path.append("../../src")

In [ ]:
import GAN.DCGAN_MNIST as dcgan

In [ ]:
if t.cuda.is_available():
    if t.cuda.device_count()>1:
        device = t.device('cuda:1')
    else:
        device = t.device('cuda')   
else:
    device = t.device('cpu')

In [ ]:
t.cuda.is_available()

In [ ]:
print(device)

## MNIST 

In [ ]:
dl_train = t.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/mnist', train=True, download=True))

dl_test  = t.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/mnist', train=False, download=True))

In [ ]:
mnist_train_data   = dl_train.dataset.data.to(dtype=t.float32)
mnist_train_labels = dl_train.dataset.targets

In [ ]:
mnist_test_data   = dl_test.dataset.data.to(dtype=t.float32)
mnist_test_labels = dl_test.dataset.targets

In [ ]:
mnist_data = np.concatenate((mnist_train_data, mnist_test_data))
mnist_labels = np.concatenate((mnist_train_labels, mnist_test_labels))

In [ ]:
index=49
plt.imshow(mnist_data[index], cmap='Greys')
plt.text(22,3,'%d' % (mnist_labels[index],), fontsize=32);

In [ ]:
mnist_data_rescaled = 2.0*mnist_data.astype('float32')/255.0 - 1.0
np.random.shuffle(mnist_data_rescaled)
mnist_data_t = t.from_numpy(mnist_data_rescaled[0:60000]).reshape(-1,1,28,28)

In [ ]:
mnist_data_t = mnist_data_t.to(device)

I am taking the whole data set. But if you run on the CPU you can consider using a smaller portion e.g 
```
mnist_data_t = t.from_numpy(mnist_data_flatened[0:20000])
```

## Models

### Discriminator

In [ ]:
discriminator = dcgan.Discriminator()
discriminator.to(device)

In [ ]:
o = discriminator(mnist_data_t[0:1])
o.shape

### Transposed convolution

<img src='figures/conv_transposed.svg' width=30% style="margin: auto; padding:20px">

$$
\begin{align}
y_0 & = w_0 x_0 + w_1 x_1 + w_2 x_2\\
y_1 & = w_0 x_1 + w_1 x_2 + w_2 x_3\\
y_2 & = w_0 x_2 + w_1 x_3 + w_2 x_4\\
\end{align}
$$

<img src='figures/conv_transposed_flip.svg' width=30% style="margin: auto; padding:20px">

$$
\begin{align}
x_0 & = w_0 y_0\\
x_1 & = w_1 y_0 + w_0 y_1\\
x_2 & = w_2 y_0 + w_1 y_1 + w_0 y_2\\
x_3 & = w_2 y_1 + w_1 y_2\\
x_4 & = w_2 y_2
\end{align}
$$

<img src='figures/conv_transposed_pad.svg' width=30% style="margin: auto; padding:20px">

### Padding

<img src='figures/conv_transposed_padded_1.svg' width=30% style="margin: auto; padding:20px">

<img src='figures/conv_transposed_padded_2.svg' width=30% style="margin: auto; padding:20px">

<img src='figures/conv_transposed_padded_3.svg' width=30% style="margin: auto; padding:20px">

### Strided

<img src='figures/conv_transposed_stride_1.svg' width=30% style="margin: auto; padding:20px">

<img src='figures/conv_transposed_stride_1.svg' width=30% style="margin: auto; padding:20px">

<img src='figures/conv_transposed_stride_2.svg' width=30% style="margin: auto; padding:20px">

<img src='figures/conv_transposed_stride_3.svg' width=30% style="margin: auto; padding:20px">

<img src='figures/conv_transposed_stride_4.svg' width=30% style="margin: auto; padding:20px">

### Generator

In [ ]:
z_dim=100

In [ ]:
generator = dcgan.Generator(z_dim)
generator.to(device)

In [ ]:
g = generator(t.rand(z_dim, device=device) )

In [ ]:
g.shape

Move data and models to the device (CUDA). 

In [ ]:
with t.no_grad():
    generator.eval()
    in_t = t.empty(1,z_dim, device=device).uniform_(-1,1)
    out_t = generator(in_t);
    plt.imshow(out_t.data.cpu().numpy().squeeze(), cmap='Greys')

Binary Cross Entropy loss with labels $l_i$ is defined as:
$$bce(\{ p_i \},\{ l_ i\}) = \frac{1}{n}\sum_{i=0}^{n-1} \left(l_i \log p_i + (1-l_i) \log(1-p_i)\right)  $$

In [ ]:
bce = t.nn.BCELoss()

In [ ]:
t.cuda.empty_cache()

In [ ]:
dataset = t.utils.data.TensorDataset(mnist_data_t)

In [ ]:
if device.type == 'cpu':
    mini_batch_size=64
else:
    mini_batch_size=2048
dataloader = t.utils.data.DataLoader(dataset, batch_size=mini_batch_size, shuffle=True)

In [ ]:
with t.no_grad():
    discriminator.eval()
    generator.eval()
    d_loss = 0
    g_loss = 0
    n_batches = 0
    for d, in dataloader:
        d_loss+= dcgan.discriminator_loss(discriminator, generator,  d, z_dim, device)
        g_loss+= dcgan.generator_loss( discriminator, generator,len(d), z_dim, device) 
        n_batches+=1
print(d_loss.item()/n_batches, g_loss.item()/n_batches)

In [ ]:
d_optimizer = t.optim.Adam(discriminator.parameters(), lr=0.0002,betas=(0.5, 0.999))
g_optimizer = t.optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))

In [ ]:
import time
start = time.time()

cols = 8
rows = 4 

fixed_noise = t.empty(cols*rows,z_dim, device=device).uniform_(-1,1)

k_discriminator = 2
k_generator = 1


n_epochs = 200
plt.ioff()
for epoch in range(1,n_epochs+1):
    discriminator.train()
    generator.train()
    for d, in dataloader:
        d_loss = dcgan.train_discriminator(discriminator, generator, d_optimizer, d, z_dim, k_discriminator, device)
        g_loss = dcgan.train_generator(discriminator, generator, g_optimizer, len(d), z_dim, k_generator, device)        
      
    if epoch % 1 == 0:
        clear_output(wait=True)
        ellapsed, remaining = dcgan.estimate(start, n_epochs, epoch)
        print('%5d %6.2f %6.2f %6.2fs %6.2fs\n' % (epoch, d_loss.item(), g_loss.item(), ellapsed, remaining))
       
    if epoch % 5 == 0:
        dcgan.gen_and_save(rows, cols, generator, fixed_noise, f"img/dcgan_{epoch:03d}.png")
            

In [ ]:
#save the generator
t.save(generator.state_dict(),"gan.pt")

In [ ]:
dcgan.gen_and_save(rows, cols, generator, fixed_noise)

In [ ]:
#Display original data
cols = 8
rows = 4 
fig, ax = plt.subplots(rows, cols, figsize=(1.5*cols,1.5*rows))
for i, j in itertools.product(range(rows), range(cols) ):
    ax[i,j].get_xaxis().set_visible(False)
    ax[i,j].get_yaxis().set_visible(False)
for i, j in itertools.product(range(rows), range(cols)):  
    ax[i,j].imshow(mnist_data[np.random.randint(0,len(mnist_data))], cmap='Greys')    

A demonstration of how to load generator.  

In [ ]:
generator_loaded = dcgan.Generator(z_dim)

In [ ]:
generator_loaded.load_state_dict(t.load('gan.pt'))
generator_loaded.eval()

In [ ]:
generator_loaded= generator_loaded.to(device)

In [ ]:
in_t = t.empty(1,z_dim, device=device).uniform_(-1,1)
out_t = generator_loaded(in_t);
plt.imshow(out_t.cpu().data.numpy().reshape(28,28), cmap='Greys');